In [5]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 4 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)

	modified:   ../MFOS (modified content, untracked content)
	deleted:    Dict/slurm-2045189.out

no changes added to commit (use "git add" and/or "git commit -a")


In [8]:
!git commit -m "big updatesss: changed functions so it nolonger chooses the first index that gives the max value"

[main b23437f] big updatesss: changed fuhnctions so it nolonger chooses the first index that gives the max value
 1 file changed, 788 deletions(-)
 delete mode 100644 Work/Dict/slurm-2045189.out


In [ ]:
git p

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm import tqdm
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import gym
from gym.spaces import Discrete, Tuple

from env_mp_2 import MetaGames
from rmax_2 import RmaxAgent, Memory

In [6]:
K_epochs = 4  # update policy for K epochs
inner_gamma = 0.8  #inner game discount factor
meta_gamma = 0.8   #meta game discount factor
R_max = 0.98
max_meta_epi = 500
max_meta_steps = 50
max_inner_epi = 10
max_inner_steps = 5

epsilon=0.2
alpha = 0.4
radius_dp =1

# creating environment
env = MetaGames("NL", "IPD")

# creating rmax agent
memory = Memory()
rmax = RmaxAgent(env, R_max, meta_gamma, max_inner_epi, max_inner_steps, radius_dp, epsilon)

#action_dim = env.d
nA = env.d
#state_dim = env.d * 2
nS = env.d * 2

#all agents' trajectory, entries will be like {[s1, a1, r1], [s2, a2, r2],...}
#torch size: {no. of inner episodes, no of inner steps, 3 (sar)}
traj = torch.empty(max_inner_epi, max_inner_steps, 3).to(device) 
our_rew = 0     #our agent's reward
oppo_rew = 0   #opponent's reward



In [8]:
traj.size()

torch.Size([10, 5, 5])

In [5]:
#initialise meta-state = empty trajectory
meta_s = torch.empty(max_inner_epi, max_inner_steps, (3+(env.num_agents-1)*2)).to(device)
memory.states.append(meta_s)

for episode in tqdm(range(max_meta_epi)): #for each meta-episode
    #reset environment
    env.reset() 
    
    for step in range(max_meta_steps):    #for each meta time step
        
        #--------------------------------------START OF INNER GAME--------------------------------------            
        #for each inner episodes            
        for epi in range(max_inner_epi):                 
            for t in range(max_inner_steps):                     #for each inner timestep
                if t == 0:
                    #initialise action by random, of size [agents, batch_size]
                    best_action = env.init_action.detach().clone()  
                else:
                    #find action that has max Q value for current state for both agents
                    best_action = env.select_action().detach().clone()     
                
                #run inner game according to that action, for K episodes & T timesteps, output used to be new_state, reward, done, _ 
                reward, info = env.step(best_action)  
                
                #trajectory only comprises of [action, reward]
                #traj[epi].append(best_action, [reward,info])   
                traj[epi,t,2:] = best_action, reward, info
                our_rew += reward.reshape(-1)
                oppo_rew += info.reshape(-1)

                #update inner r matrix
                env.innerr[0, best_action[0]] = reward
                env.innerr[1, best_action[1]] = info

                env.innerq[0, best_action[0]] = env.innerr[0, best_action[0]] + inner_gamma * torch.max(env.innerq[0, best_action[0]]) 
                env.innerq[1, best_action[1]] = env.innerr[1, best_action[1]] + inner_gamma * torch.max(env.innerq[1, best_action[1]])
                
                inner_state = best_action
        
        #---------------------------------------END OF INNER GAME---------------------------------          
        
        #meta-action = inner game Q table for our agent
        meta_a = env.innerq[0,:]
        memory.actions.append(meta_a)
        
        #meta-reward = sum of inner rewards of our agent over K episodes & T timesteps
        meta_r = torch.round(our_rew, decimals = radius_dp)             
        memory.rewards.append(meta_r)
        
        #next-meta-state = trajectorty
        next_meta_s = torch.round(traj, decimals = radius_dp).long().to(device)               
        memory.states.append(next_meta_s)
        
        #Rmax update step
        rmax.update(env, Memory, meta_s, meta_a, next_meta_s)
        


  0%|                                                                                            | 0/500 [00:00<?, ?it/s]


TypeError: type list doesn't define __round__ method

In [22]:
torch.argmax(env.innerq, 1)

tensor([0, 1], device='cuda:0')

In [5]:
env.init_action.detach().clone()

tensor([0, 0], device='cuda:0')

In [ ]:
        #find discretized state index that corresponds to the real state value
        dstate_index=[]
        for j in torch.round(state, decimals = radius_dp):
            for i,x in enumerate(ref_arr):
                if x==j:
                    dstate_index.append(i)